In [126]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sympy import symbols, solve

# Functions

In [127]:
def getMinorEntry(matrix, row, col):
    n = matrix.shape[0]
    minor = np.zeros((n-1, n-1), dtype=object)
    rowIdx = 0
    for i in range(n):
        colIdx = 0
        if (i != row):
            for j in range(n):
                if (j != col):
                    minor[rowIdx, colIdx] = matrix[i, j]
                    colIdx += 1
            rowIdx += 1
    
    return getDeterminant(minor)

def getDeterminant(matrix):
    if (matrix.shape[0] == 1):
        return matrix[0, 0]
    elif (matrix.shape[0] == 2):
        return(matrix[0, 0]*matrix[1, 1] - (matrix[0, 1]*matrix[1, 0]))
    else:
        determinant = 0
        sign = 1
        for j in range(matrix.shape[1]):
            determinant += matrix[0, j]*getMinorEntry(matrix, 0, j)*sign
            sign *= -1;
    return determinant

def getEigenVal(matrix):
    mcopy = matrix.copy()
    x = symbols('x')
    nshape = mcopy.shape[0]
    lambdaId = np.eye(nshape)*x - mcopy

    expr = getDeterminant(lambdaId)
    val = solve(expr)

    # Ambil value unik saja
    val = [np.float(x) for x in set(val)]
    val.sort(reverse=True)
    return val

In [128]:
def getIdxUtama(matrix, row):
    col = 0
    while (col < matrix.shape[1]):
        if (matrix[row, col] != 0):
            return col
        else:
            col += 1
    return -1

def swapRow(matrix, row1, row2):
    for j in range(matrix.shape[1]):
        tmp = matrix[row1, j]
        matrix[row1, j] = matrix[row2, j]
        matrix[row2, j] = tmp

def gaussElimination(matrix):
    mtemp = matrix.copy()
    row = mtemp.shape[0]
    col = mtemp.shape[1]
    zeroRow = row - 1
    for k in range(row):
        minIdx = k
        for i in range(k, row):
            if (getIdxUtama(mtemp, i) == -1):
                minIdx = i
            elif (getIdxUtama(mtemp, i) < getIdxUtama(mtemp, minIdx)):
                minIdx = i
        swapRow(mtemp, k, minIdx)

        if (getIdxUtama(mtemp, k) == -1):
            swapRow(mtemp, k, zeroRow)
            zeroRow -= 1

        idxUtama = getIdxUtama(mtemp, k)
        if (idxUtama != -1):
            divisor = mtemp[k, idxUtama]
            for j in range(col):
                mtemp[k, j] = mtemp[k, j] / divisor

        for i in range(k + 1, row):
            mult = mtemp[i, idxUtama]
            for j in range(col):
                mtemp[i, j] -= mult * mtemp[k, j]
    
    return mtemp

def gaussJordanElimination(matrix):
    mtemp = gaussElimination(matrix)
    row = mtemp.shape[0]
    col = mtemp.shape[1]
    for k in range(row - 1, -1, -1):
        idxUtama = getIdxUtama(mtemp, k)
        
        for i in range(k - 1, -1, -1):
            mult = mtemp[i, idxUtama]
            for j in range(col):
                mtemp[i, j] -= mult * mtemp[k, j]

    return mtemp

# Singular Value Decomposition 

### Important functions:

### * np.zeros -> make n x m matrix filled with 0
### * np.fill_diagonal -> fill diagonal without changing matrix shape
### * symbol() and solve() in sympy -> solving equation

In [129]:
mat = np.array([[3, 1, 1],
                [-1, 3, 1]], dtype=float)
mshape = mat.shape
mat

array([[ 3.,  1.,  1.],
       [-1.,  3.,  1.]])

In [130]:
#Langkah 1: Singular Kiri
sleft = np.matmul(mat, mat.T)
sright = np.matmul(mat.T, mat)
sright

array([[10.,  0.,  2.],
       [ 0., 10.,  4.],
       [ 2.,  4.,  2.]])

In [131]:
#Eigen value
print(np.linalg.eigvals(sright), np.linalg.eigvals(sleft))
print(getEigenVal(sright), getEigenVal(sleft))

[ 0. 10. 12.] [12. 10.]
[12.0, 10.0, 0.0] [12.0, 10.0]


In [132]:
def getEigenMatrix(smat):
    eigenList = getEigenVal(smat)
    resultMatrix = []
    for ctr in range(len(eigenList)):
        eigval = eigenList[ctr]
        mdiag = np.eye(smat.shape[0])*eigval
        mtemp = mdiag - smat
        mtemp = gaussJordanElimination(mtemp)

        eigenVec = [0 for i in range(mtemp.shape[0])]
        delta = 0
        for i in range(len(eigenVec) - 1, -1 , -1):
            idxUtama = getIdxUtama(mtemp, i)
            
            # Kalo terlompat, diisi 1
            # Misal tadi isi x3 (idx 2) lalu x1 (idx 0) -> x2 kosong jadi assign 1
            if (i != len(eigenVec) - 1):
                delta = prevFilled - idxUtama - 1
                if (delta > 0):
                    while (delta != 0):
                        eigenVec[prevFilled - delta] = 1.0
                        delta -= 1

            if (idxUtama == -1):
                eigenVec[i] = 1.0
                prevFilled = i
            else:
                eigenVec[idxUtama] = 0
                prevFilled = idxUtama
                for j in range(mtemp.shape[1]):
                    if (j != idxUtama):
                        eigenVec[idxUtama] -= mtemp[idxUtama, j] * eigenVec[j]

        # Normalize vectors
        tmpList = [math.pow(x, 2) for x in eigenVec]
        vectorLength = math.sqrt(sum(tmpList))
        eigenVec = [x/vectorLength for x in eigenVec]

        resultMatrix.append(eigenVec)

    resultMatrix = np.array(resultMatrix, dtype=float)

    return resultMatrix.T

In [133]:
# VTVec
getEigenMatrix(sright).T

array([[ 0.40824829,  0.81649658,  0.40824829],
       [-0.89442719,  0.4472136 ,  0.        ],
       [-0.18257419, -0.36514837,  0.91287093]])

In [134]:
# UVec
getEigenMatrix(sleft)

array([[ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678]])